In [33]:
from pathlib import Path # Manejo de rutas multiplataforma
from dvc.repo import Repo # Maneja el repositorio DVC
from __future__ import annotations # Permite anotaciones de tipo diferidas
from typing import Dict, Tuple, Optional # Tipado opcional para mayor claridad
from dvc.api import get_url, open as dvc_open # Abre o resuelve archivos versionados

import os # Manejo de rutas y variables del sistema
import yaml # Lectura de archivos .dvc (YAML)
import hashlib # Calcula hashes MD5 para verificar integridad
import pandas as pd # Lectura y manejo de datos tabulares
import configparser

In [34]:
def ensure_repo_ready(repo_root: str = "/work") -> None:
    """
    Verifica 
    - Que `repo_root` sea una carpeta válida de proyecto con Git y DVC.
    - Que exista el directorio `repo_root`.
    - Que tenga un subdirectorio `.git` (es un repo Git).
    - Que tenga un subdirectorio `.dvc` (es un repo DVC).

    Lanza:
    - FileNotFoundError si `repo_root` no existe.
    - RuntimeError si falta `.git` o `.dvc`.
    """
    if not os.path.isdir(repo_root):
        raise FileNotFoundError(f"Repo root no existe: {repo_root}")
    if not os.path.isdir(os.path.join(repo_root, ".git")):
        raise RuntimeError(f"No es un repo Git: {repo_root}")
    if not os.path.isdir(os.path.join(repo_root, ".dvc")):
        raise RuntimeError(f"No es un repo DVC: {repo_root} (.dvc no encontrado)")


def _md5_file(path: str, chunk_size: int = 1024 * 1024) -> str:
    """
    Calcula el hash MD5 de un archivo leyendo para verificar integridad contra el valor 
    guardado por DVC en el puntero `.dvc` cuando se usa la cache por defecto basada en md5.
    
    Parámetros:
    - path: ruta absoluta al archivo.
    - chunk_size: tamaño de cada bloque de lectura en bytes (por defecto 1 MB).

    Retorna:
    - Cadena hex MD5 del contenido del archivo.
    """
    h = hashlib.md5()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(chunk_size), b""):
            h.update(chunk)
    return h.hexdigest()


def _read_expected_md5_from_dvc(pointer_path: str) -> Optional[str]:
    """
    Lee el MD5 esperado desde un archivo puntero `.dvc`.

    Formato `.dvc`:
      - md5: <hash>
      - hash: md5
      - path: <nombre_del_archivo>

    Parámetros:
    - pointer_path: ruta absoluta al archivo `.dvc`.

    Retorna:
    - El hash MD5 (str) si existe, o None si el puntero no existe / no trae md5.

    Uso:
    - Permite comparar el MD5 esperado por DVC con el MD5 real del archivo local.
    """
    if not os.path.exists(pointer_path):
        return None
    with open(pointer_path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f) or {}
    outs = data.get("outs") or []
    if not outs:
        return None
    out = outs[0]
    return out.get("md5") or out.get("checksum") or None


def dvc_get_resolved_url(path_repo_rel: str, repo_root: str = "/work") -> str:
    """
    Resuelve la URL remota del output versionado (por ejemplo `s3://.../<hash>`).

    Parámetros:
    - path_repo_rel: ruta repo-relativa del archivo versionado.
    - repo_root: raíz del repo.

    Retorna:
    - URL a la ubicación del blob en el remoto configurado por DVC.

    Útil para:
    - Depurar que DVC “ve” el archivo y está correctamente trackeado.
    - Confirmar a qué objeto remoto S3 apuntaría una lectura por DVC.

    Nota:
    - Soporta outputs definidos en `.dvc` de archivo único (single-file stage),
      donde `outs.path` es relativo al directorio del `.dvc`.
    """
    # intento 1: ruta repo-relativa completa (caso general)
    try:
        return get_url(path_repo_rel, repo=repo_root)
    except Exception:
        # intento 2: single-file .dvc (repo=subcarpeta, path=basename)
        subdir = os.path.dirname(path_repo_rel)          # p.ej. "data/raw"
        basename = os.path.basename(path_repo_rel)       # p.ej. "work_absenteeism_modified.csv"
        return get_url(basename, repo=os.path.join(repo_root, subdir))


def dvc_read_csv_verified(
    path_repo_rel: str,
    repo_root: str = "/work",
    prefer_dvc: bool = False,
    verify_local_md5: bool = True,
    pandas_read_csv_kwargs: Optional[Dict] = None,
) -> Tuple[pd.DataFrame, str]:
    """
    Lee un CSV versionado con DVC garantizando integridad cuando consultas local.

    Estrategia:
    - Si `prefer_dvc=True`: siempre lee vía DVC (máxima fidelidad). Devuelve ("dvc").
    - Si `prefer_dvc=False`:
        1) Si existe el archivo local y `verify_local_md5=True`, compara MD5 local
           con el MD5 esperado del puntero `.dvc`. Si coincide -> lee local (rápido).
        2) Si no existe o no coincide, hace fallback a `dvc_open()` (lee de cache/remoto).
           Devuelve ("dvc").

    Parámetros:
    - path_repo_rel: ruta repo-relativa del CSV.
    - repo_root: raíz del repo.
    - prefer_dvc: fuerza lectura por DVC (ignora archivo local materializado).
    - verify_local_md5: si True, valida MD5 local antes de confiar en lectura local.
    - pandas_read_csv_kwargs: dict con kwargs para `pandas.read_csv()` (sep, encoding, etc.).

    Retorna:
    - (df, source) donde source ∈ {"local", "dvc"} para saber de dónde se leyó.

    Excepciones:
    - Propaga errores de lectura en caso de que el archivo no exista ni localmente ni
      en el remoto o haya problemas de credenciales.

    Nota:
    - En notebooks de equipo y CI, este método ayuda a detectar drift local (archivos
      tocados fuera de DVC) y asegura consistencia con lo versionado.
    """
    ensure_repo_ready(repo_root)
    if pandas_read_csv_kwargs is None:
        pandas_read_csv_kwargs = {}

    local_path = os.path.join(repo_root, path_repo_rel)
    dvc_pointer = local_path + ".dvc"  # p.ej. data/raw/file.csv.dvc

    # Opción: forzar lectura por DVC siempre (máxima fidelidad/reproducibilidad)
    if prefer_dvc:
        with dvc_open(path_repo_rel, repo=repo_root, mode="rb") as f:
            return pd.read_csv(f, **pandas_read_csv_kwargs), "dvc"

    # Lectura local con verificación MD5 (si aplica)
    if os.path.exists(local_path):
        if verify_local_md5:
            expected = _read_expected_md5_from_dvc(dvc_pointer)
            if expected:
                try:
                    if _md5_file(local_path) == expected:
                        return pd.read_csv(local_path, **pandas_read_csv_kwargs), "local"
                except Exception:
                    # Si hay cualquier problema, hacemos fallback a DVC
                    pass
            # Si no hay puntero .dvc o no trae md5, cae a DVC para garantizar integridad
        else:
            # Si no quieres verificar, lee local directo
            return pd.read_csv(local_path, **pandas_read_csv_kwargs), "local"

    # Fallback robusto: lectura vía DVC (usa cache o remoto)
    with dvc_open(path_repo_rel, repo=repo_root, mode="rb") as f:
        return pd.read_csv(f, **pandas_read_csv_kwargs), "dvc"


In [35]:
# Parámetros editables de lectura del dataset

# Ruta ABSOLUTA a la raíz del repo dentro del contenedor.
# Docker se monta en /work. Si se cambia el compose,
# ajusta REPO_ROOT.
REPO_ROOT = "/work" # Ruta donde esta montado el Repo.
PATH = "data/raw/work_absenteeism_modified.csv" # Ruta repo-relativa del dataset versionado con DVC:

# Argumentos para pandas.read_csv. Opcional delimita separadores, codificación, etc.
READ_KW: Dict = {}  # e.g.: {"sep": ",", "encoding": "utf-8"}

# Modo de lectura:
# - PREFER_DVC=True  -> SIEMPRE leer vía DVC (usa cache/remoto). Máxima fidelidad, un poco más de overhead.
# - PREFER_DVC=False -> Intenta leer local si el archivo existe y pasa verificación de MD5; si no, cae a DVC.
PREFER_DVC = False          # True => siempre dvc_open (máxima fidelidad)
VERIFY_LOCAL_MD5 = True     # True => valida MD5 local vs .dvc antes de lectura local

# =========================
# Inspección de entorno + demo de lectura
# =========================
print("Repo root:", REPO_ROOT, "| existe:", Path(REPO_ROOT).exists())
print("CSV esperado:", PATH)
try:
    # dvc_get_resolved_url resuelve el objeto remoto (p. ej. s3://.../<hash>)
    # Si falla aquí, puede ser: puntero .dvc ausente/incorrecto, credenciales AWS faltantes,
    # o que el archivo no está trackeado por DVC.
    print("URL DVC:", dvc_get_resolved_url(PATH, repo_root=REPO_ROOT))
except Exception as e:
    # Evita bloquear el flujo si no puedes resolver la URL en este momento;
    # igual intentaremos leer con dvc_read_csv_verified que hace fallback apropiado.
    print("No se pudo resolver URL DVC:", e.__class__.__name__, str(e)[:120], "...")

# --- Lectura robusta con verificación de integridad ---
# dvc_read_csv_verified hace:
#   1) Si PREFER_DVC=True -> lee por DVC sí o sí.
#   2) Si PREFER_DVC=False:
#        - Si el archivo existe localmente y VERIFY_LOCAL_MD5=True:
#            compara MD5 local contra el MD5 del puntero .dvc.
#            * Si coincide -> lee local (rápido).
#            * Si no coincide o algo falla -> cae a DVC (lee desde cache/remoto).
#        - Si el archivo no existe localmente -> lee por DVC.
df, source = dvc_read_csv_verified(
    PATH,
    repo_root=REPO_ROOT,
    prefer_dvc=PREFER_DVC,
    verify_local_md5=VERIFY_LOCAL_MD5,
    pandas_read_csv_kwargs=READ_KW,
)

print(f"Leído desde: {source} | filas={len(df)} | columnas={len(df.columns)}")

Repo root: /work | existe: True
CSV esperado: data/raw/work_absenteeism_modified.csv
No se pudo resolver URL DVC: OutputNotFoundError Unable to find DVC file with output 'work_absenteeism_modified.csv' ...
Leído desde: local | filas=754 | columnas=22


## Nota:

* Si alguna vez Git en el contenedor advierte “dubious ownership”: "**git config --global --add safe.directory /work**"

* Para evitar sorpresas, mantén data/raw/*.csv en .gitignore y versiona solo los *.dvc.

### Para no corromper kernels ni perder el entorno.

1. Guarda Notebook o "**ctrl+s**"
2. Cierra el kernel limpio:
   * Menú → Kernel → Shut Down Kernel
   * Luego: File → Close and Shutdown Notebook
3. Cierra la pestaña del navegador.
4. Ve a la terminal donde lo ejecutaste.
5. Presiona Ctrl + C (detiene el servidor).
6. Si te pregunta “Shutdown this notebook server (y/[n])?”, escribe y.

### Salir del docker y sincronizar notebook
1. exit
2. docker compose down
3. git add notebooks/EDA/eda_V1.ipynb
4. git commit -m "feat: análisis inicial EDA"
5. git push

In [15]:
df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours,mixed_type_col
0,11.0,26.0,7.0,3.0,1.0,289.0,36.0,13.0,33.0,239.554,...,1.0,2.0,1.0,0.0,1.0,90.0,172.0,30.0,4.0,535
1,36.0,0.0,7.0,3.0,1.0,118.0,13.0,18.0,50.0,239.554,...,1.0,1.0,1.0,0.0,0.0,98.0,178.0,31.0,0.0,584
2,3.0,23.0,7.0,4.0,1.0,179.0,51.0,18.0,38.0,239.554,...,1.0,0.0,1.0,0.0,0.0,89.0,170.0,31.0,2.0,249
3,7.0,7.0,7.0,5.0,1.0,279.0,5.0,14.0,39.0,239.554,...,1.0,2.0,1.0,1.0,0.0,68.0,168.0,24.0,4.0,538
4,11.0,23.0,7.0,65.0,1.0,289.0,36.0,13.0,33.0,239.554,...,1.0,2.0,1.0,0.0,1.0,90.0,172.0,30.0,2.0,85
